# Working with Data - Computer Lab for Guest Lecture Julia Lane

In this computer lab we will learn more more details and practice data work to enhance the content of the lecture presented by Julia Lane on responsible data use. We will address a research question, think about data and measurement errors, and manipulate data. 

OUTLINE: 
1. Define a research question 
2. Think about what data are available 
3. Think about possible measurement errors 
4. Think about the interpretation of your results 
5. Inform your results by linking datasets 

# 1. Define a reserach question
Which Community Districts in NYC show the highest number of complaints?

# 2. Think about what data are available
Find suitable data by searching the CUSP Data Catalog https://datahub.cusp.nyu.edu/catalog. You can use Urban Profiler to investigate the Metadata associated with each dataset. Using this tool will help you to decide which attributes of the data you need to answer your question so you don't have to load the entire dataset. 

In [3]:
import os
import pandas as pd
import numpy as np
import re
PUIdata = os.getenv('PUIDATA')
dfdata = os.getenv("DFDATA")

In [2]:
# Load dataset

comp = pd.read_csv('/projects/open/NYCOpenData/nycopendata/data/erm2-nwe9/1446832678/erm2-nwe9', \
                            usecols=['Agency','Agency Name','Borough','Complaint Type','Community Board',\
                                     'Incident Zip','Location','Status','Unique Key', 'Resolution Description'])

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 3. Think about possible measurement errors
Do you see any problems regarding possible measurement error? Think about who is represented in the data, ommissions, duplications, content error, missing data, etc. 

In [3]:
# How many unique values do we have? 
print('Unique Boroughs Count: ', len(comp.Borough.unique()))
print('Unique Boroughs list: ', comp.Borough.unique())

print('Unique Community Boards: ', len(comp['Community Board'].unique()))
print('Unique Boroughs list: ', comp['Community Board'].unique())

('Unique Boroughs Count: ', 6)
('Unique Boroughs list: ', array(['BROOKLYN', 'Unspecified', 'STATEN ISLAND', 'MANHATTAN', 'BRONX',
       'QUEENS'], dtype=object))
('Unique Community Boards: ', 77)
('Unique Boroughs list: ', array(['13 BROOKLYN', '0 Unspecified', '18 BROOKLYN', '12 BROOKLYN',
       '02 STATEN ISLAND', '08 MANHATTAN', '06 BROOKLYN', '10 BRONX',
       '06 MANHATTAN', '04 BROOKLYN', '04 QUEENS', '09 MANHATTAN',
       '14 BROOKLYN', '05 MANHATTAN', '02 MANHATTAN', '10 QUEENS',
       '07 QUEENS', '02 QUEENS', '11 BROOKLYN', '05 QUEENS', '05 BROOKLYN',
       '09 QUEENS', '12 MANHATTAN', '01 BROOKLYN', '11 MANHATTAN',
       '03 BRONX', '03 STATEN ISLAND', '08 BROOKLYN', '03 BROOKLYN',
       '07 MANHATTAN', '10 MANHATTAN', '04 BRONX', '01 MANHATTAN',
       '07 BRONX', '09 BRONX', '03 MANHATTAN', '13 QUEENS', '02 BRONX',
       '12 QUEENS', '16 BROOKLYN', '08 QUEENS', '04 MANHATTAN',
       '14 QUEENS', '10 BROOKLYN', '11 QUEENS', '06 BRONX', '08 BRONX',
       '05 BRON

In [ ]:
# Why do we have so many? Some of them are unspecified, missing. Some might be invalid entries. 
# We should have 59 Community Districts.
comp.columns = [c.replace(' ', '_') for c in comp.columns]
comp.count()

comp = (comp[comp.Borough.str.contains("Unspecified") == False])
comp = (comp[comp.Community_Board.str.contains("Unspecified") == False])

#Remove the lines that have fake districts numbers
fake = ['64', '81', '55', '83', '28', '80', '26', '95', '82', '27', '56', '84']
for i in range(len(fake)):
    comp = (comp[comp.Community_Board.str.contains(fake[i]) == False]) 

print('Unique Boroughs Count: ', len(comp.Borough.unique()))
print('Unique Community Boards: ', len(comp['Community_Board'].unique()))
print('Unique Boroughs list: ', comp['Community_Board'].unique())

In [5]:
# Check for duplicates? Are these plausible?
# The duplicates are likely multiple calls for a single complaint
comp.columns = [c.replace(' ', '_') for c in comp.columns]
dupes = comp[comp.duplicated(['Unique_Key'])]
print ('Complaint Types Duplicates: ', dupes.Complaint_Type.value_counts())
comp.drop_duplicates(keep='last', inplace=True)

('Complaint Types Duplicates: ', Rodent                            20
Unsanitary Animal Pvt Property     2
Name: Complaint_Type, dtype: int64)


In [6]:
# What about missing values? Can you detect any patterns? 
null_data = comp[comp.isnull().any(axis=1)]
print(len(null_data))
null_data.head(30)

3078315


,Unique_Key,Agency,Agency_Name,Complaint_Type,Incident_Zip,Status,Resolution_Description,Community_Board,Borough,Location
0,31911011,DOT,Department of Transportation,Street Condition,11224,Open,NaN,13 BROOKLYN,BROOKLYN,"(40.57343122248129, -73.99174247588253)"
26,31910446,DOT,Department of Transportation,Traffic Signal Condition,11385,Open,NaN,05 QUEENS,QUEENS,"(40.7004993066336, -73.90020149091094)"
28,31913287,DOT,Department of Transportation,Traffic Signal Condition,11385,Open,NaN,05 QUEENS,QUEENS,"(40.701968198847794, -73.88049664749286)"
31,31914364,DOT,Department of Transportation,Traffic Signal Condition,11415,Open,NaN,09 QUEENS,QUEENS,"(40.7093480453081, -73.82968656053104)"
78,31915575,DOT,Department of Transportation,Street Light Condition,11212,Open,NaN,16 BROOKLYN,BROOKLYN,"(40.663392865381105, -73.91399726503518)"
79,31908183,DOT,Department of Transportation,Street Condition,11436,Open,NaN,12 QUEENS,QUEENS,"(40.66706786430971, -73.79306643645447)"
90,31910712,DOT,Department of Transportation,Street Light Condition,11367,Open,NaN,08 QUEENS,QUEENS,"(40.71580414459643, -73.82447553099988)"
102,31911025,DOT,Department of Transportation,Street Condition,10011,Open,NaN,04 MANHATTAN,MANHATTAN,"(40.744249943905174, -73.99396600722591)"
109,31914004,DEP,Department of Environmental Protection,Noise,10001,Open,NaN,04 MANHATTAN,MANHATTAN,"(40.75326104677971, -74.00382228632859)"
113,31914950,DEP,Department of Environmental Protection,Noise,10001,Open,NaN,04 MANHATTAN,MANHATTAN,NaN


Any report that doesn't have a location has several missing values, all that are related to location: incident zip, community board, borough, and location.  Cases that are open so not have a valid resolution descriptor, since they have yet to be resolved.

In [7]:
# Data Editing: Is it possible to replace missing values? Is it possible to use Complaint Type 
# to fill missings in Resolution Descriptor?
#This could be possible if you knew the relationship between complaint type and the resolution descriptor.
#Is the resolution description NaN because there was no resolution?

null_data[null_data.Resolution_Description.isnull()].Complaint_Type.value_counts()

Water System                               338608
Street Light Condition                     308274
Noise                                      220386
Sewer                                      207840
Dirty Conditions                           201461
General Construction/Plumbing              168490
Sanitation Condition                       156709
Building/Use                               151652
Traffic Signal Condition                   146757
Street Condition                           135577
Missed Collection (All Materials)          106775
Graffiti                                    90722
Derelict Vehicles                           71988
Snow                                        54900
Elevator                                    46795
Air Quality                                 38681
Blocked Driveway                            36988
Special Enforcement                         31683
Other Enforcement                           30053
Damaged Tree                                26030


In [ ]:
# Generate marker for unplausible Community Districts
# How do these districts look like? 
# Drop the marked districts

In [8]:
# Produce your result: Generate an indicator which ranks the Community District by complaint numbers 
# on the Community district level
comp.Community_Board.value_counts()

12 MANHATTAN        268053
12 QUEENS           229383
03 BROOKLYN         197306
07 QUEENS           195961
17 BROOKLYN         191720
05 BROOKLYN         189765
14 BROOKLYN         187794
07 BRONX            185729
01 BROOKLYN         185057
01 STATEN ISLAND    182713
04 BRONX            181953
01 QUEENS           171484
05 QUEENS           168570
12 BRONX            168455
13 QUEENS           163851
05 BRONX            161078
18 BROOKLYN         156100
07 MANHATTAN        151419
09 QUEENS           151242
12 BROOKLYN         150320
03 MANHATTAN        150296
08 MANHATTAN        149076
09 BRONX            147457
15 BROOKLYN         146129
05 MANHATTAN        142028
08 BROOKLYN         141119
11 BROOKLYN         139890
04 BROOKLYN         139706
10 MANHATTAN        137344
03 STATEN ISLAND    136487
09 BROOKLYN         135833
02 MANHATTAN        133860
04 MANHATTAN        132660
10 QUEENS           128116
09 MANHATTAN        127565
08 QUEENS           123131
11 BRONX            122081
0

In [ ]:
# Safe reduced data frame (Community District level)

# 4. Think about the interpretation of your results?
What do you have to keep in mind when interpreting your results? Are they generable? Does the way the data is collected influence your results? To better inform city agancies it might be good to explore in more detail the underlying dempgraphics/infrastructure of a Community District becasue this might influence 311 calls. You can do this by merging external data on the Community District level to your analysis data. 

In [9]:
# Population by Community District
df_pop = pd.read_csv('/projects/open/NYCOpenData/nycopendata/data/xi7c-iiu2/1414245891/xi7c-iiu2')
df_pop.tail(30)

,Borough,CD Number,CD Name,1970 Population,1980 Population,1990 Population,2000 Population,2010 Population
29,Brooklyn,18,"Canarsie, Flatlands",188643,169092,162428,194653,193543
30,Manhattan,1,"Battery Park City, Tribeca",7706,15918,25366,34420,60978
31,Manhattan,2,"Greenwich Village, Soho",84337,87069,94105,93119,90016
32,Manhattan,3,"Lower East Side, Chinatown",181845,154848,161617,164407,163277
33,Manhattan,4,"Chelsea, Clinton",83601,82164,84431,87479,103245
34,Manhattan,5,Midtown Business District,31076,39544,43507,44028,51673
35,Manhattan,6,"Stuyvesant Town, Turtle Bay",122465,127554,133748,136152,142745
36,Manhattan,7,"West Side, Upper West Side",212422,206669,210993,207699,209084
37,Manhattan,8,Upper East Side,200851,204305,210880,217063,219920
38,Manhattan,9,"Manhattanville, Hamilton Heights",113606,103038,106978,111724,110193


In [10]:
# Check variables in file
df_pop.columns = [c.replace(' ', '_') for c in df_pop.columns]
print (df_pop.Borough.value_counts())
print (df_pop.CD_Number.value_counts())

Brooklyn         18
Queens           14
Manhattan        12
Bronx            12
Staten Island     3
Name: Borough, dtype: int64
1     5
2     5
3     5
8     4
4     4
5     4
6     4
7     4
9     4
10    4
11    4
12    4
13    2
14    2
17    1
15    1
16    1
18    1
Name: CD_Number, dtype: int64


In [4]:
df_demo = pd.read_csv(PUIdata + "/Final_Demographics.csv")
df_demo.head()

,FIPS,cd_id,Total Population,Population Density (per sq. mile),% Total Population: Male,% Total Population: 18 to 24 Years,% Total Population: 25 to 34 Years,% Total Population: 35 to 44 Years,% Population 5 Years And Over: Speak Only English,% Population 5 Years And Over: Spanish or Spanish Creole,...,"Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations","% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations",% Employed Civilian Population 16 Years And Over: Service occupations,% Employed Civilian Population 16 Years And Over: Sales and office occupations,"% Employed Civilian Population 16 Years And Over: Farming, fishing, and forestry occupations","% Employed Civilian Population 16 Years And Over: Construction, extraction, and maintenance occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations"
0,3603701,BX08,106737,31229.95006,46.65,10.73,15.04,11.32,46.80,39.24,...,665,1518,42.66,28.95,20.89,0.24,2.65,4.60,1.40,3.20
1,3603702,BX12,134644,19966.67839,46.35,11.35,14.29,12.57,73.09,18.19,...,1156,4174,29.57,33.98,20.40,0.00,7.08,8.97,1.95,7.02
2,3603703,BX10,121209,12913.81703,45.20,8.62,13.74,12.78,61.79,26.43,...,941,3433,36.20,22.85,25.09,0.00,7.68,8.18,1.76,6.42
3,3603704,BX11,135839,35677.95453,50.09,8.10,17.43,14.09,43.22,36.45,...,2189,5592,30.06,27.86,22.24,0.00,7.03,12.81,3.60,9.20
4,3603705,BX03,172247,39405.79222,44.72,14.24,14.89,12.38,36.82,54.24,...,1437,5436,16.80,41.00,22.29,0.03,8.45,11.43,2.39,9.04


In [12]:
# How many community districts are in file? 
len(df_pop)

59

** There are 59 rows in this dataframe, so there are the correct number of community districts **

In [13]:
# Manipulate data to get some information on demographics by Community District. 
# Think about who might be more likely to call 311
df_demo= df_demo[[u'FIPS', u'cd_id', u'Total Population',
       u'% Total Population: 18 to 24 Years',
       u'% Total Population: 25 to 34 Years',
       u'% Total Population: 35 to 44 Years', u'Population 25 Years and over:',
       u'% Population 25 Years and over: Less Than High School',
       u'% Population 25 Years and over: High School Graduate or more (includes equivalency)',
       u'% Population 25 Years and over: Some college or more',
       u'% Population 25 Years and over: Bachelor\'s degree or more',
       u'% Population 25 Years and over: Master\'s degree or more',
       u'% Population 25 Years and over: Professional school degree or more',
       u'Employed civilian Population 16 Years and over:',
       u'Median household income (In 2014 Inflation Adjusted Dollars)']]
df_demo.columns

Index([u'FIPS', u'cd_id', u'Total Population',
       u'% Total Population: 18 to 24 Years',
       u'% Total Population: 25 to 34 Years',
       u'% Total Population: 35 to 44 Years', u'Population 25 Years and over:',
       u'% Population 25 Years and over: Less Than High School',
       u'% Population 25 Years and over: High School Graduate or more (includes equivalency)',
       u'% Population 25 Years and over: Some college or more',
       u'% Population 25 Years and over: Bachelor's degree or more',
       u'% Population 25 Years and over: Master's degree or more',
       u'% Population 25 Years and over: Professional school degree or more',
       u'Employed civilian Population 16 Years and over:',
       u'Median household income (In 2014 Inflation Adjusted Dollars)'],
      dtype='object')

In [14]:
df_demo.head()

,FIPS,cd_id,Total Population,% Total Population: 18 to 24 Years,% Total Population: 25 to 34 Years,% Total Population: 35 to 44 Years,Population 25 Years and over:,% Population 25 Years and over: Less Than High School,% Population 25 Years and over: High School Graduate or more (includes equivalency),% Population 25 Years and over: Some college or more,% Population 25 Years and over: Bachelor's degree or more,% Population 25 Years and over: Master's degree or more,% Population 25 Years and over: Professional school degree or more,Employed civilian Population 16 Years and over:,Median household income (In 2014 Inflation Adjusted Dollars)
0,3603701,BX08,106737,10.73,15.04,11.32,71928,16.93,83.07,63.82,39.87,18.66,7.24,47433,54224
1,3603702,BX12,134644,11.35,14.29,12.57,88219,19.78,80.22,48.97,23.44,6.20,0.24,59430,44906
2,3603703,BX10,121209,8.62,13.74,12.78,85902,17.34,82.66,52.74,27.17,10.43,1.58,53487,54962
3,3603704,BX11,135839,8.10,17.43,14.09,93326,22.84,77.16,49.08,24.11,10.90,3.39,60765,47910
4,3603705,BX03,172247,14.24,14.89,12.38,98478,38.02,61.98,35.49,11.87,3.84,1.26,60127,22041


In [15]:
# Save data frame
df_demo.sort('Median household income (In 2014 Inflation Adjusted Dollars)').head(10)\
[['Median household income (In 2014 Inflation Adjusted Dollars)', \
 'FIPS', 'cd_id', 'Total Population']]

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,Median household income (In 2014 Inflation Adjusted Dollars),FIPS,cd_id,Total Population
7,20872,3603707,BX05,132850
10,21116,3603710,BX01,167147
11,21116,3603710,BX02,167147
4,22041,3603705,BX03,172247
5,22041,3603705,BX06,172247
33,25259,3604007,BK16,123772
8,27203,3603708,BX04,141467
44,29578,3604018,BK13,110727
15,31340,3603804,MN11111,129713
34,32996,3604008,BK05,155863


** Community districts in the Bronx have the lowest median household income.  Is there a relationship between household income and the liklihood to make 311 complaints? **

In [16]:
df_demo.sort('Median household income (In 2014 Inflation Adjusted Dollars)', ascending=False).head(10)\
[['Median household income (In 2014 Inflation Adjusted Dollars)', \
 'FIPS', 'cd_id', 'Total Population']]

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Median household income (In 2014 Inflation Adjusted Dollars),FIPS,cd_id,Total Population
22,120190,3603810,MN01,159903
23,120190,3603810,MN02,159903
16,115238,3603805,MN08,221898
31,113045,3604005,BK06,125062
17,106181,3603806,MN07,192374
20,105625,3603808,MN06,143349
18,98561,3603807,MN05,149447
19,98561,3603807,MN04,149447
30,84568,3604004,BK02,139070
24,82718,3603901,SI03,170741


** For the most part ommunity districts in Manhattan have the highest median household incomes.  Two brooklyn community districts and one staten island community district are in the top ten median household incomes. **

In [17]:
df_demo.sort('% Population 25 Years and over: Professional school degree or more').head(10)\
[['% Population 25 Years and over: Professional school degree or more', \
 'FIPS', 'cd_id', 'Total Population']]

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,% Population 25 Years and over: Professional school degree or more,FIPS,cd_id,Total Population
1,0.24,3603702,BX12,134644
34,0.31,3604008,BK05,155863
9,0.46,3603709,BX09,190126
8,0.60,3603708,BX04,141467
11,0.69,3603710,BX02,167147
10,0.69,3603710,BX01,167147
33,0.75,3604007,BK16,123772
7,0.84,3603707,BX05,132850
36,0.95,3604010,BK17,137526
56,0.98,3604112,QN12,251002


** The community districts with the lowest percentage of residents (25 and older) with a professional school degree or more are mostly located in the Bronx (with one Queens district at number 10).  Do neighborshoods with lower education levels make fewer 311 calls?

In [18]:
df_demo.sort('% Population 25 Years and over: Professional school degree or more', ascending=False).head(10)\
[['% Population 25 Years and over: Professional school degree or more', \
 'FIPS', 'cd_id', 'Total Population']]

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,% Population 25 Years and over: Professional school degree or more,FIPS,cd_id,Total Population
17,18.46,3603806,MN07,192374
16,16.72,3603805,MN08,221898
20,13.58,3603808,MN06,143349
22,12.98,3603810,MN01,159903
23,12.98,3603810,MN02,159903
31,11.56,3604005,BK06,125062
18,8.97,3603807,MN05,149447
19,8.97,3603807,MN04,149447
52,8.37,3604108,QN06,107893
30,8.27,3604004,BK02,139070


In [19]:
# Infrastructure by Community District
df_infr = pd.read_csv(PUIdata + "/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv")

In [20]:
# Check variables in file
df_infr.columns

Index([u'FIPS', u'Geographic Identifier', u'Qualifying Name', u'Households',
       u'Households: With An Internet Subscription',
       u'Households: Dial-Up Alone', u'Households: Dsl',
       u'Households: With Mobile Broadband',
       u'Households: Without Mobile Broadband', u'Households: Cable Modem',
       u'Households: With Mobile Broadband.1',
       u'Households: Without Mobile Broadband.1', u'Households: Fiber-Optic',
       u'Households: With Mobile Broadband.2',
       u'Households: Without Mobile Broadband.2',
       u'Households: Satellite Internet Service',
       u'Households: With Mobile Broadband.3',
       u'Households: Without Mobile Broadband.3',
       u'Households: Two or More Fixed Broadband Types, or Other',
       u'Households: With Mobile Broadband.4',
       u'Households: Without Mobile Broadband.4',
       u'Households: Mobile Broadband Alone or With Dialup',
       u'Households: Internet Access Without A Subscription',
       u'Households: No Internet Acc

In [21]:
# How many community districts are in file? 
len(df_infr) 

59

In [22]:
# Manipulate data to get some information on internet/broadband useage by Community District
# Aggregate the mobile subscription data
df_infr['mobile_aggr'] = df_infr[u'Households: With Mobile Broadband'] + \
    df_infr[u'Households: With Mobile Broadband.4'] +\
    df_infr[u'Households: With Mobile Broadband'] +\
    df_infr[u'Households: With Mobile Broadband.1'] +\
    df_infr[u'Households: With Mobile Broadband.2'] +\
    df_infr[u'Households: With Mobile Broadband.3'] +\
    df_infr[u'Households: With Mobile Broadband.4'] +\
    df_infr[u'Households: Mobile Broadband Alone or With Dialup']

In [23]:
# Aggregate internet type by high and low connections
df_infr['high_speed'] = df_infr[u'Households: Dsl'] +\
    df_infr[u'Households: With Mobile Broadband'] +\
    df_infr[u'Households: Cable Modem'] +\
    df_infr[u'Households: With Mobile Broadband.1'] +\
    df_infr[u'Households: Fiber-Optic'] +\
    df_infr[u'Households: With Mobile Broadband.2'] +\
    df_infr[u'Households: With Mobile Broadband.3'] +\
    df_infr[u'Households: Two or More Fixed Broadband Types, or Other'] +\
    df_infr[u'Households: With A Broadband Internet Subscription']

df_infr['low_speed'] = df_infr[u'Households: Dial-Up Alone'] + df_infr[u'Households: Satellite Internet Service']

In [24]:
# Save data frame 
df_infr.head()

,FIPS,Geographic Identifier,Qualifying Name,Households,Households: With An Internet Subscription,Households: Dial-Up Alone,Households: Dsl,Households: With Mobile Broadband,Households: Without Mobile Broadband,Households: Cable Modem,...,% Households: With An Internet Subscription,Households.1,Households: Has A Computer,Households: With Dial-Up Internet Subscription Alone,Households: With A Broadband Internet Subscription,Households: Without An Internet Subscription,Households: No Computer,mobile_aggr,high_speed,low_speed
0,3603701,79500US3603701,"NYC-Bronx Community District 8--Riverdale, New...",42035,31795,444,2813,946,1867,19178,...,75.64,42035,35048,404,30943,3701,6987,21983,71900,519
1,3603702,79500US3603702,"NYC-Bronx Community District 12--Wakefield, Ne...",44830,32243,178,849,405,444,18653,...,71.92,44830,36700,178,31435,5087,8130,13965,70781,309
2,3603703,79500US3603703,"NYC-Bronx Community District 10--Co-op City, N...",47050,32729,158,1863,398,1465,20044,...,69.56,47050,38700,158,32333,6209,8350,15912,72240,158
3,3603704,79500US3603704,NYC-Bronx Community District 11--Pelham Parkwa...,44922,32003,141,1478,474,1004,17917,...,71.24,44922,37237,122,31278,5837,7685,14360,70442,449
4,3603705,79500US3603705,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,35503,88,1297,651,646,15847,...,61.68,57556,42576,88,33408,9080,14980,23703,75391,282


# 5. Inform your results by linking datasets
Now you want to link the three data frames to produce summary statistics for Community Districts which show a high number of complaints vs. Community Districts which show a lower number of complaints. Please keep in mind that the identifiers used for the linkage (Community Dostrict IDs) should be recored the same way. Use regular expressions to harmonize the identifiers if possible. The identifiers should look like BK01, BK02, etc.
https://docs.python.org/2/library/re.html

In [4]:
comp.head()

,Unique Key,Agency,Agency Name,Complaint Type,Incident Zip,Status,Resolution Description,Community Board,Borough,Location
0,31911011,DOT,Department of Transportation,Street Condition,11224,Open,NaN,13 BROOKLYN,BROOKLYN,"(40.57343122248129, -73.99174247588253)"
1,31908754,CHALL,CHALL,Opinion for the Mayor,NaN,Email Sent,Your comments have been submitted to the Mayor...,0 Unspecified,Unspecified,NaN
2,31910423,DPR,Department of Parks and Recreation,Root/Sewer/Sidewalk Condition,11234,Open,The Department of Parks and Recreation usually...,18 BROOKLYN,BROOKLYN,"(40.62500363580505, -73.92072558378698)"
3,31909924,NYPD,New York City Police Department,Illegal Parking,11218,Assigned,Your complaint has been received by the Police...,12 BROOKLYN,BROOKLYN,"(40.634522428879706, -73.97479041437481)"
4,31913310,HRA,HRA Benefit Card Replacement,Benefit Card Replacement,NaN,Closed,The Human Resources Administration received yo...,0 Unspecified,Unspecified,NaN


In [ ]:
# Harmonize identifier of dataframe 1
comp['cd_id'] = np.ones(len(comp), np.float)

for i in range (len(comp)):
    try:
        ct = (comp['Community_Board'][i]).split()[0]
        if (comp['Community_Board'][i]).split()[1] == 'BRONX':
            boro = 'BX'
        if (comp['Community_Board'][i]).split()[1] == 'BROOKLYN':
            boro = 'BK'
        if (comp['Community_Board'][i]).split()[1] == 'QUEENS':
            boro = 'QN'
        if (comp['Community_Board'][i]).split()[1] == 'MANHATTAN':
            boro = 'MN'
        if (comp['Community_Board'][i]).split()[1] == 'STATEN ISLAND':
            boro = 'SI'
    except: (KeyError)

In [5]:
# Harmonize identifier of dataframe 2
df_demo.head()

,FIPS,cd_id,Total Population,Population Density (per sq. mile),% Total Population: Male,% Total Population: 18 to 24 Years,% Total Population: 25 to 34 Years,% Total Population: 35 to 44 Years,% Population 5 Years And Over: Speak Only English,% Population 5 Years And Over: Spanish or Spanish Creole,...,"Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations","% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations",% Employed Civilian Population 16 Years And Over: Service occupations,% Employed Civilian Population 16 Years And Over: Sales and office occupations,"% Employed Civilian Population 16 Years And Over: Farming, fishing, and forestry occupations","% Employed Civilian Population 16 Years And Over: Construction, extraction, and maintenance occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations"
0,3603701,BX08,106737,31229.95006,46.65,10.73,15.04,11.32,46.80,39.24,...,665,1518,42.66,28.95,20.89,0.24,2.65,4.60,1.40,3.20
1,3603702,BX12,134644,19966.67839,46.35,11.35,14.29,12.57,73.09,18.19,...,1156,4174,29.57,33.98,20.40,0.00,7.08,8.97,1.95,7.02
2,3603703,BX10,121209,12913.81703,45.20,8.62,13.74,12.78,61.79,26.43,...,941,3433,36.20,22.85,25.09,0.00,7.68,8.18,1.76,6.42
3,3603704,BX11,135839,35677.95453,50.09,8.10,17.43,14.09,43.22,36.45,...,2189,5592,30.06,27.86,22.24,0.00,7.03,12.81,3.60,9.20
4,3603705,BX03,172247,39405.79222,44.72,14.24,14.89,12.38,36.82,54.24,...,1437,5436,16.80,41.00,22.29,0.03,8.45,11.43,2.39,9.04


In [ ]:
# Harmonize identifier of dataframe 3

df_infr['BoardNum'] = np.ones(len(df_infr), np.float)
for i in range (0, len(df_infr)):
    df_infr['BoardNum'][i] = str(df_infr['Qualifying Name'][i]).split('District ')[1].split('--')[0] + " " + str(df_infr['Qualifying Name'][i]).split('-')[1].split(' ')[0]
#code taken from Sofiya Elyukin

df_infr['valid'] = np.ones(len(df_infr), np.float)
for i in range (0, len(df_infr)):
    try:
        if int(df_infr['BoardNum'][i].split(' &')[0]) < 10:
            df_infr['valid'][i] = 'False'
    except ValueError:
        if int(df_infr['BoardNum'][i].split(' ')[0]) < 10:
            df_infr['BoardNum'][i] = "0" + df_infr['BoardNum'][i]
            df_infr['valid'][i] = "True"
        else: df_infr['valid'][i] = "True"
#code taken from Sofiya Elyukin

for i in range (0, len(df_infr)):
    if not df_infr['valid'][i] == 'True':
        if (i-1)/2 == int(i/2):
            df_infr.BoardNum[i] = '0' + str(df_infr.BoardNum[i].split('&')[0]) + str(df_infr.BoardNum[i].split('&')[1].split()[1])
        else: 
            df_infr.BoardNum[i] = '0' + str(df_infr.BoardNum[i].split('&')[1].strip())
df_infr[['BoardNum', 'valid']]

In [ ]:
df_infr['cd_id'] = np.ones(len(df_infr), np.float)

for i in range (len(df_infr)):
    ct = (df_infr['BoardNum'][i]).split()[0]
    if (df_infr['BoardNum'][i]).split()[1] == 'Bronx':
        boro = 'BX'
    if (df_infr['BoardNum'][i]).split()[1] == 'Brooklyn':
        boro = 'BK'
    if (df_infr['BoardNum'][i]).split()[1] == 'Queens':
        boro = 'QN'
    if (df_infr['BoardNum'][i]).split()[1] == 'Manhattan':
        boro = 'MN'
    if (df_infr['BoardNum'][i]).split()[1] == 'Staten':
        boro = 'SI'
    df_infr['cd_id'][i] =  boro + ct
print (df_infr[['BoardNum', 'cd_id']])

In [ ]:
# Link the 3 dataframes
comp_inf = pd.merge(comp, df_infr, how='left', on='cd_id')
dem_inf_comp = pd.merge(comp_inf, demo, how='left', on='cd_id'

In [ ]:
# Are the demographics and infrastructure different in Community Districts that show more complaints than others?